#forked from https://github.com/Azure/cognitive-search-vector-pr

# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK
## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [ ]:
! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv

## Import required libraries and environment variables

In [3]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential(key)

In [ ]:
print(openai.api_base)

## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

Add the Transcript file to data folder and update the file path and execute

In [6]:
# Generate Document Embeddings using OpenAI Ada 002

# Read the text-sample.json
with open('../data/text-sample.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

#@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings


# Generate embeddings for title and content fields
for item in input_data:
    #title = item['title']
    content = item['content']
    #title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    #item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings

# Output embeddings to docVectors.json file
with open("../output/docVectors.json", "w") as f:
    json.dump(input_data, f)

## Create your search index
Create your search index schema and vector search configuration:

In [7]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


 videoindex created


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [8]:
# Upload some documents to the index
with open('../output/docVectors.json', 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 623 documents


## Perform a vector similarity search

In [9]:
# Pure Vector Search
query = "tell me about Davinci GPT 3.0 model"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vectors= [vector],
    select=[ "content" ],
)  
  
for result in results:  
    
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    


Score: 0.9488607
Content: Davinci GPT 3.0 model here.
Score: 0.868408
Content: So we are going to look at the GPT 3.0 models and and in the bottom.
Score: 0.8666922
Content: We have GPD 3 models generating and understanding text.


## Perform a Hybrid Search

In [11]:
# Hybrid Search
query = "responsible"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=["content"],
    top=3
)  
  
for result in results:  
    
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
   


Score: 0.03306011110544205
Content: And the focus is on responsible AI, ethical AI as well.
Score: 0.03306011110544205
Content: I was testing the our government's responsible AI document.
Score: 0.016129031777381897
Content: Thank.


## Perform a Semantic Hybrid Search

In [10]:
# Semantic Hybrid Search
query = "what is azure search?"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  

results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    select=[ "content"],
    query_type="semantic", query_language="en-us", semantic_configuration_name='my-semantic-config', query_caption="extractive", query_answer="extractive",
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    
    print(f"Content: {result['content']}")
  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Content: And then we can use that into Azure Cognitive Search to build a search index.
Caption: And then we can use that into<em> Azure</em> Cognitive<em> Search</em> to build a search index..

Content: If your application is more general purpose like sales offers or build creation or some kind of marketing, you could generally use what comes out-of-the-box.
Caption: If your application is more general purpose like sales offers or build creation or some kind of marketing, you could generally use what comes out-of-the-box..

Content: As you can see, they are deployed within your Azure subscription.
Caption: As you can see, they are deployed within your<em> Azure subscription..</em>

